<a href="https://colab.research.google.com/github/INLANA01/capstone_project_coref_with_transformers/blob/main/Neural_Coref_Model_C_and_M.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook
import spacy

In [2]:
MODEL_URL = "https://github.com/huggingface/neuralcoref-models/releases/" \
            "download/en_coref_md-3.0.0/en_coref_md-3.0.0.tar.gz"

In [ ]:
# MODEL_URL2 = "https://github.com/google-research-datasets/gap-coreference.git"

In [3]:
! pip uninstall spacy

Uninstalling spacy-2.2.4:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.7/dist-packages/bin/*
    /usr/local/lib/python3.7/dist-packages/spacy-2.2.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/spacy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/bin/theano_cache.py
    /usr/local/lib/python3.7/dist-packages/bin/theano_nose.py
Proceed (y/n)? y
  Successfully uninstalled spacy-2.2.4


In [4]:
!pip install murmurhash3

  Created wheel for murmurhash3: filename=murmurhash3-2.3.5-cp37-cp37m-linux_x86_64.whl size=26488 sha256=dd99330f59f171c9ed7c98c64291dec48498292bc74c54714f6589690c81005c
  Stored in directory: /root/.cache/pip/wheels/06/eb/e4/f57324cd9c1bf001c9ba6d6926ad5231eca80964ed80b3610e
Successfully built murmurhash3


In [5]:
# Current version of neuralcoref library needs spacy version 2.2.4 or older
!pip install spacy==2.2.4

     |████████████████████████████████| 10.6MB 4.4MB/s 


In [3]:
!pip install {MODEL_URL}

     |████████████████████████████████| 161.3MB 57kB/s 
  Created wheel for en-coref-md: filename=en_coref_md-3.0.0-cp37-cp37m-linux_x86_64.whl size=163495544 sha256=274134777b6dbf42947317b278811c471a1893ab8062b10aa04a7d24bd434b4e
  Stored in directory: /root/.cache/pip/wheels/aa/a3/8f/9df13c027b75169bcca62682563e9823bb213c72a2cc3efed8
Successfully built en-coref-md


# Using the hugging face implementation of nuralcoref: https://github.com/huggingface/neuralcoref

In [5]:
!pip install neuralcoref

     |████████████████████████████████| 286kB 4.1MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 7.5MB 58.5MB/s 
     |████████████████████████████████| 81kB 12.1MB/s 
  Created wheel for boto3: filename=boto3-1.17.68-py2.py3-none-any.whl size=128921 sha256=38f7e639ecb3210379460a25c0dc467cf282c108152a3965518afe66c2ac579b
  Stored in directory: /root/.cache/pip/wheels/7b/3c/28/e64c294edb631f00138ef6e094406471b6a1aed91ccd30fa52
Successfully built boto3
ERROR: botocore 1.20.68 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [8]:
!pip3 install spacy
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [9]:
nlp = spacy.load("en_core_web_sm")

In [2]:
# import en_coref_md
# nlp = en_coref_md.load()
# !python -m spacy download en

import spacy
# nlp = spacy.load("en")
nlp = spacy.load("en_core_web_sm")

In [3]:
# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp)

## Check if the mention, if within the range of reference

In [ ]:
def is_inside(offset, span):
    return offset >= span[0] and offset <= span[1]

def is_a_mention_of(sent, pron_offset, entity_offset_a, entity_offset_b):
    doc = nlp(sent)
    if doc._.has_coref:
        for cluster in doc._.coref_clusters:
            main = cluster.main
            main_span = main.start_char, main.end_char
            mentions_spans = [(m.start_char, m.end_char) for m in cluster.mentions \
                              if (m.start_char, m.end_char) != main_span]
            if is_inside(entity_offset_a, main_span) and \
                    np.any([is_inside(pron_offset, s) for s in mentions_spans]):
                return "A"
            elif is_inside(entity_offset_b, main_span) and \
                    np.any([is_inside(pron_offset, s) for s in mentions_spans]):
                return "B"
            else:
                return "NEITHER"
    else:
        return "NEITHER"

In [ ]:
# test_sent = "The doctor came in. She held a paper in her hand."
entity_offset_a = test_sent.index("doctor")
entity_offset_b = test_sent.index("paper")
pron_offset = test_sent.index("She")

is_a_mention_of(test_sent, pron_offset, entity_offset_a, entity_offset_b)

'A'

In [ ]:
 ! git clone https://github.com/google-research-datasets/gap-coreference.git

Cloning into 'gap-coreference'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19
Unpacking objects: 100% (19/19), done.


In [ ]:
gap_train = pd.read_csv("https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv", delimiter='\t', index_col="ID")
gap_train.head()

,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
ID,,,,,,,,,,
test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [ ]:
def predict(df):
    pred = pd.DataFrame(index=df.index, columns=["A", "B", "NEITHER"]).fillna(False)
    for i, row in tqdm_notebook(df.iterrows()):
        pred.at[i, is_a_mention_of(row["Text"], row["Pronoun-offset"], row["A-offset"], row["B-offset"])] = True
    return pred

In [ ]:
train_preds = predict(gap_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
gap_train["NEITHER"] = np.logical_and(~gap_train["A-coref"], ~gap_train["B-coref"])

In [ ]:
gap_train[["A-coref", "B-coref", "NEITHER"]].describe()

,A-coref,B-coref,NEITHER
count,2000,2000,2000
unique,2,2,2
top,False,False,False
freq,1126,1075,1799


In [ ]:
train_preds.describe()

,A,B,NEITHER
count,2000,2000,2000
unique,2,2,2
top,False,False,True
freq,1875,1931,1806


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(gap_train[["A-coref", "B-coref", "NEITHER"]], train_preds[["A", "B", "NEITHER"]]))

              precision    recall  f1-score   support

           0       0.70      0.10      0.18       874
           1       0.58      0.04      0.08       925
           2       0.11      0.97      0.19       201

   micro avg       0.16      0.16      0.16      2000
   macro avg       0.46      0.37      0.15      2000
weighted avg       0.59      0.16      0.13      2000
 samples avg       0.16      0.16      0.16      2000



In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(gap_train[["A-coref", "B-coref", "NEITHER"]], train_preds[["A", "B", "NEITHER"]]))

0.161
